In [2]:
import advent

def parse_line(line):
    words = line.split(' ')
    return tuple([int(words[ix]) for ix in [6, 12, 18, 21, 27, 30]])

lines = advent.get_lines(19, map_fn=parse_line)
# Example: 
# Blueprint 1:
# Each ore robot costs 4 ore. 
# Each clay robot costs 2 ore. 
# Each obsidian robot costs 3 ore and 14 clay. 
# Each geode robot costs 2 ore and 7 obsidian.


In [3]:
tadd3 = lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2])
tadd4 = lambda a, b: (a[0] + b[0], a[1] + b[1], a[2] + b[2], a[3] + b[3])

def try_to_build(blueprint, resources, miners, resources_size=3):
    # Checks if a suggested number of miners is possible to build with the given resources
    # and if so, return tuple with resources left after building
    ore_needed = (miners[0] * blueprint[0]) + (miners[1] * blueprint[1]) + (miners[2] * blueprint[2]) + (miners[3] * blueprint[4])
    clay_needed = (miners[2] * blueprint[3])
    obsidian_needed = (miners[3] * blueprint[5])
    if ore_needed > resources[0] or clay_needed > resources[1] or obsidian_needed > resources[2]:
        return False
    if resources_size == 3:
        return (resources[0] - ore_needed, resources[1] - clay_needed, resources[2] - obsidian_needed)
    elif resources_size == 4:
        return (resources[0] - ore_needed, resources[1] - clay_needed, resources[2] - obsidian_needed, resources[3])
    raise ValueError("resources_size must be 3 or 4")

In [5]:
# idea 1: good ol' graph path optimization!
from functools import lru_cache

cache = {}

@lru_cache(maxsize=None)
def maximum_geodes(blueprint, resources=(0,0,0), miners=(1,0,0,0), minute=1, ore_phase=True):
    # resources: ore, clay, obsidian
    # miners: ore, clay, obsidian, geode
    # minute: we are at the 'start' of this minute (no resources collected yet, nothing built)
    # returns how many geodes we can get at the end of minute 24
    if minute == 24: return miners[3] # can only mine
    # The possible actions are: build 1 or more miners for each of the 4 types
    max_ore = resources[0] // blueprint[0]
    max_clay = resources[0] // blueprint[1]
    max_obsidian = min(resources[0] // blueprint[2], resources[1] // blueprint[3])
    max_geode = min(resources[0] // blueprint[4], resources[2] // blueprint[5])
    possible_results = []

    for ore in range(max_ore+1):
        for clay in range(max_clay+1):
            for obsidian in range(max_obsidian+1):
                for geode in range(max_geode+1):
                    to_build = (ore, clay, obsidian, geode)
                    #if max(max_ore, max_clay, max_obsidian, max_geode) > 0 and to_build == (0, 0, 0, 0): continue
                    resources_left = try_to_build(blueprint, resources, to_build)
                    if resources_left:
                        resources_left = tadd3(resources_left, miners[:3]) # mine
                        new_miners = tadd4(miners, to_build) # build
                        subresult = maximum_geodes(blueprint, resources_left, new_miners, minute+1)
                        possible_results.append(subresult)
    #print(minute, max_b)
    #cache[(resources, miners, minute)] = max(possible_results)
    return miners[3] + max(possible_results) # also mine geodes

#maximum_geodes(lines[1])
# Takes too long :(

In [34]:
# Attempt 2: choose the next robot to build, from the list of 4. Then build that next robot ASAP, before generating another robot
# hopefully, we will build maximum like 6 robots, so this will generate 4^6 = 4096 paths, which seems managable
# and many of those paths will be cut off early, such as any path that starts with 'geode' (because you don't have obsidian yet)
# unfortunately in practice we build way more robots than just 6

def amax(*args):
    return max(args, key=lambda x: x[0])

def maximum_geodes_2(blueprint, resources=(0,0,0,0), miners=(1,0,0,0), minute=1, to_build=(0,0,0,0), next_build=None):
    # max_build = 0: ore, 1: clay, 2: obsidian, 3: geode
    if minute == 24: return miners[3] + resources[3], []
    if next_build == 2 and miners[1] == 0 and to_build[1] == 0: return 0, []
    if next_build == 3 and miners[2] == 0 and to_build[2] == 0: return 0, []
    if (miners[0] + to_build[0]) > 10: return -1, [] # maximum 10 ore miners allowed
    if (miners[1] + to_build[1]) > 10: return -1, [] # maximum 10 clay miners allowed

    if next_build is None:
        return amax(
            maximum_geodes_2(blueprint, resources, miners, minute, to_build, next_build=0),
            maximum_geodes_2(blueprint, resources, miners, minute, to_build, next_build=1)
        )

    to_build_tmp = (int(next_build==0), int(next_build==1), int(next_build==2), int(next_build==3))
    resources_left = try_to_build(blueprint, resources, to_build_tmp, resources_size=4)
    #print(resources, miners, minute, resources_left, to_build)
    if resources_left:
        # (tentatively) build the robot, then continue at same timestamp
        to_build_new = tadd4(to_build, to_build_tmp)
        next_build_options = {0: [0, 1], 1:[1, 2], 2: [1, 2, 3], 3: [3]}[next_build]
        return max([maximum_geodes_2(blueprint, resources_left, miners, minute, to_build_new, i) for i in next_build_options], key=lambda x: x[0])
    else:
        # pass the time: mine resources, then build all outstanding robots, then add 1 to minute
        new_miners = tadd4(miners, to_build)
        new_resources = tadd4(resources, miners)
        max_geodes, subresult = maximum_geodes_2(blueprint, new_resources, new_miners, minute + 1, (0, 0, 0, 0), next_build)
        return max_geodes, [(new_miners, new_resources, minute+1)] + subresult

blueprint1 = (4, 2, 3, 14, 2, 7)
blueprint2 = (2, 3, 3, 8, 3, 12)
maximum_geodes_2(blueprint2, next_build=0)
# also takes too long :((

(14,
 [((1, 0, 0, 0), (1, 0, 0, 0), 2),
  ((1, 0, 0, 0), (2, 0, 0, 0), 3),
  ((2, 0, 0, 0), (1, 0, 0, 0), 4),
  ((2, 0, 0, 0), (3, 0, 0, 0), 5),
  ((3, 0, 0, 0), (3, 0, 0, 0), 6),
  ((4, 0, 0, 0), (4, 0, 0, 0), 7),
  ((6, 0, 0, 0), (4, 0, 0, 0), 8),
  ((6, 1, 0, 0), (7, 0, 0, 0), 9),
  ((6, 3, 0, 0), (7, 1, 0, 0), 10),
  ((6, 5, 0, 0), (7, 4, 0, 0), 11),
  ((6, 7, 0, 0), (7, 9, 0, 0), 12),
  ((6, 9, 0, 0), (7, 16, 0, 0), 13),
  ((6, 9, 2, 0), (7, 9, 0, 0), 14),
  ((6, 10, 3, 0), (7, 10, 2, 0), 15),
  ((6, 10, 4, 0), (10, 12, 5, 0), 16),
  ((6, 10, 5, 0), (13, 14, 9, 0), 17),
  ((6, 10, 6, 0), (16, 16, 14, 0), 18),
  ((6, 10, 8, 1), (13, 10, 8, 0), 19),
  ((6, 10, 8, 1), (19, 20, 16, 1), 20),
  ((6, 10, 8, 2), (22, 30, 12, 2), 21),
  ((6, 10, 8, 3), (25, 40, 8, 4), 22),
  ((6, 10, 8, 3), (31, 50, 16, 7), 23),
  ((6, 10, 8, 4), (34, 60, 12, 10), 24)])

In [37]:
len([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3])

28

In [48]:
# attempt 3: without (as much?) recursion?
# Generate an order of robots, e.g. [0, 0, 0, 1, 1, 1, 2, 2, 3]
# which means: ore, ore, ore, clay, clay, etc, then simulate it
# After minute 24, save the result
# the problem with this is, you probably have to build like 12 or more robots for many blueprints,
# which is 16 million combinations

from itertools import product
from tqdm.notebook import tqdm

# todo there very well may be more than 10 robots in prod
max_robots = 12


# blueprint = (4, 2, 3, 14, 2, 7)
blueprint = (2, 3, 3, 8, 3, 12)
robot_orders = product(*[[0, 1, 2, 3]] * max_robots)

max_geodes = -1
max_build_order = None

cache = {}

robot_orders = [[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3]]

for order in tqdm(robot_orders):
    
    build_ix, minute = 0, 1
    resources = (0,0,0,0)
    miners = (1,0,0,0)

    to_build_this_minute = (0,0,0,0)

    while minute < 25:
        #if build_ix >= len(order): break

        #if tuple(order[:(build_ix+1)]) in cache:
            # this is because e.g. if we calculate the geodes for [0,0,0,0,0,0] and [0,0,0,0,0,1],we already have the subresult for [0,0,0,0,0] saved and just have to do the last step
        #    minute, resources, miners, to_build_this_minute = cache[tuple(order[:(build_ix+1)])]
        #    build_ix += 1
        #    continue

        next_build = order[build_ix]

        to_build_tmp = (int(next_build==0), int(next_build==1), int(next_build==2), int(next_build==3))
        resources_left = try_to_build(blueprint, resources, to_build_tmp, 4)
        if resources_left:
            to_build_this_minute = tadd4(to_build_this_minute, to_build_tmp)
            resources = resources_left
            # Now add the robot_order so far to the cache
            cache[tuple(order[:(build_ix+1)])] = (minute, resources, miners, to_build_this_minute)

            build_ix += 1
        else:
            # pass the turn
            resources = tadd4(resources, miners)
            miners = tadd4(miners, to_build_this_minute)
            print("Beginning of turn", minute+1, "with miners", miners, "and resources", resources, "Done building:", to_build_this_minute)
            to_build_this_minute = (0,0,0,0)
            minute += 1

    if resources[3] > max_geodes:
        max_geodes = resources[3]
        max_build_order = order

print(max_geodes, max_build_order)


  0%|          | 0/1 [00:00<?, ?it/s]

Beginning of turn 2 with miners (1, 0, 0, 0) and resources (1, 0, 0, 0) Done building: (0, 0, 0, 0)
Beginning of turn 3 with miners (1, 0, 0, 0) and resources (2, 0, 0, 0) Done building: (0, 0, 0, 0)
Beginning of turn 4 with miners (2, 0, 0, 0) and resources (1, 0, 0, 0) Done building: (1, 0, 0, 0)
Beginning of turn 5 with miners (2, 0, 0, 0) and resources (3, 0, 0, 0) Done building: (0, 0, 0, 0)
Beginning of turn 6 with miners (3, 0, 0, 0) and resources (3, 0, 0, 0) Done building: (1, 0, 0, 0)
Beginning of turn 7 with miners (4, 0, 0, 0) and resources (4, 0, 0, 0) Done building: (1, 0, 0, 0)
Beginning of turn 8 with miners (6, 0, 0, 0) and resources (4, 0, 0, 0) Done building: (2, 0, 0, 0)
Beginning of turn 9 with miners (6, 1, 0, 0) and resources (7, 0, 0, 0) Done building: (0, 1, 0, 0)
Beginning of turn 10 with miners (6, 3, 0, 0) and resources (7, 1, 0, 0) Done building: (0, 2, 0, 0)
Beginning of turn 11 with miners (6, 5, 0, 0) and resources (7, 4, 0, 0) Done building: (0, 2, 0, 0

In [ ]:
# Attempt 4:

# Work backwards instead to calculate the 'optimal move', with the goal of getting 1 geode (or n geodes)

def optimal_next_build(blueprint, resources, miners, to_build):
    next_resources = tadd4(resources, miners)
    next_miners = tadd4(miners, to_build)

resources = (0,0,0,0)
miners = (1,0,0,0)
minute = 1
to_build_this_minute = (0,0,0,0)

blueprint = (2, 3, 3, 8, 3, 12)

while minute < 25:

    next_build = optimal_next_build(blueprint, resources, miners, to_build_this_minute)

    to_build_tmp = (int(next_build==0), int(next_build==1), int(next_build==2), int(next_build==3))
    resources_left = try_to_build(blueprint, resources, to_build_tmp, 4)
    if resources_left:
        to_build_this_minute = tadd4(to_build_this_minute, to_build_tmp)
        resources = resources_left
    else:
        # pass the turn
        resources = tadd4(resources, miners)
        miners = tadd4(miners, to_build_this_minute)
        to_build_this_minute = (0,0,0,0)
        minute += 1
